# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г.

In [334]:
import re
import typing as t
from collections import defaultdict
from functools import lru_cache
from pathlib import Path

import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk.corpus import stopwords, wordnet
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset, random_split

In [335]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [336]:
DATA_DIR = Path("data/")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE.upper()} device")

Using CUDA device


In [337]:
def on_cuda(device: str) -> bool:
    return device == "cuda"


def common_train(
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        train_dataloader: DataLoader,
        epochs: int,
        test_dataloader: DataLoader = None,
        lr_scheduler=None,
        verbose: int = 100,
        device: str = "cpu",
) -> t.List[float]:
    train_losses = []
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n" + "-" * 32)
        train_loss = train_loop(
            train_dataloader,
            model,
            loss_fn,
            optimizer,
            verbose=verbose,
            device=device,
        )
        train_losses.append(train_loss.item())
        if test_dataloader:
            loss, acc = test_loop(test_dataloader, model, loss_fn, device=device)
            if lr_scheduler:
                lr_scheduler.step(loss)
        torch.cuda.empty_cache()
    return train_losses


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = 100,
        device: str = "cpu",
) -> torch.Tensor:
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss = 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss
        if batch % verbose == 0:
            print(f"loss: {loss:>7f}  [{batch * len(x):>5d}/{size:>5d}]")

        del x, y, pred, loss
        torch.cuda.empty_cache()

    return avg_loss / num_batches


@torch.no_grad()
def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss, correct = 0, 0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        avg_loss += loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # noqa

        del x, y, pred
        torch.cuda.empty_cache()

    avg_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {accuracy:>4f}, Avg loss: {avg_loss:>8f} \n")

    return avg_loss, accuracy


def train_test_split(dataset: t.Union[Dataset, t.Sized], train_part: float) -> t.Tuple[Subset, Subset]:
    train_size = round(train_part * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, lengths=(train_size, test_size))
    return train_dataset, test_dataset


@torch.no_grad()
def get_y_test_y_pred(
        model: nn.Module,
        test_dataloader: DataLoader,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    y_test = []
    y_pred = []
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        y_test.append(y)
        y_pred.append(pred)

        del x
        torch.cuda.empty_cache()

    return torch.hstack(y_test).detach().cpu(), torch.hstack(y_pred).detach().cpu()

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [338]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [339]:
text = text.lower()

alphabet = list(set(nltk.word_tokenize(text.replace(".", ""))))
word2index = {w: i for i, w in enumerate(alphabet)}

first_sentence = nltk.sent_tokenize(text)[0].replace(".", "")
[word2index[w] for w in nltk.word_tokenize(first_sentence)]

[7, 2, 16, 0, 3, 4, 6, 18]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [340]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [341]:
text = text.lower()

alphabet = list(set(nltk.word_tokenize(text.replace(".", ""))))
word2index = {w: i for i, w in enumerate(alphabet)}

first_sentence = nltk.sent_tokenize(text)[0].replace(".", "")
words = nltk.word_tokenize(first_sentence)

vectors = torch.zeros(len(words), len(alphabet))
indices = [(i, word2index[w]) for i, w in enumerate(words)]
vectors[list(zip(*indices))] = 1
vectors

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.]])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [342]:
torch.manual_seed(0)

# https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
# кажется, что просят невозможного. Попадание в размерность - уже успех
embeds = nn.Embedding(num_embeddings=len(alphabet), embedding_dim=len(alphabet))
indices = torch.tensor([word2index[w] for w in nltk.word_tokenize(first_sentence)])
embeds(indices)

tensor([[-1.3962e+00, -6.6144e-02, -3.5836e-01, -1.5616e+00, -3.5464e-01,
          1.0811e+00,  1.3148e-01,  1.5735e+00,  7.8143e-01, -1.0787e+00,
         -7.2091e-01,  1.4708e+00,  2.7564e-01,  6.6678e-01, -9.9439e-01,
         -1.1894e+00, -1.1959e+00, -5.5963e-01,  5.3347e-01,  4.0689e-01,
          3.9459e-01,  1.7151e-01,  8.7604e-01, -2.8709e-01,  1.0216e+00],
        [ 1.0554e+00,  1.7784e-01, -2.3034e-01, -3.9175e-01,  5.4329e-01,
         -3.9516e-01, -4.4622e-01,  7.4402e-01,  1.5210e+00,  3.4105e+00,
         -1.5312e+00, -1.2341e+00,  1.8197e+00, -5.5153e-01, -5.6925e-01,
          9.1997e-01,  1.1108e+00,  1.2899e+00, -1.4782e+00,  2.5672e+00,
         -4.7312e-01,  3.3555e-01, -1.6293e+00, -5.4974e-01, -4.7983e-01],
        [-7.6447e-01,  2.4084e-01,  1.6643e-01, -2.2318e+00,  1.3892e+00,
         -5.0233e-01,  1.6797e+00, -1.0240e+00,  1.6859e+00, -1.2177e+00,
          7.6496e-01,  1.1971e+00, -7.1279e-01, -6.5576e-02,  2.2050e+00,
          1.7852e+00, -1.1840e-02,  

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

In [343]:
surnames_df = pd.read_csv(DATA_DIR / "surnames.csv")
surnames_df.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


2.2 Закодировать национальности числами, начиная с 0.

In [344]:
surnames_labeler = LabelEncoder()
surnames_df["target"] = surnames_labeler.fit_transform(surnames_df["nationality"])
print(f"classes: {len(surnames_labeler.classes_)}")
surnames_df.head()

classes: 18


,surname,nationality,target
0,Woodford,English,4
1,Coté,French,5
2,Kore,English,4
3,Koury,Arabic,0
4,Lebzak,Russian,14


2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`


In [345]:
class Vocab:
    pad = "<PAD>"

    def __init__(self, series: pd.Series):
        uniques = set()
        max_len = 0
        for w in map(str.lower, series):
            uniques.update(w)
            max_len = max(len(w), max_len)

        self.alphabet = [self.pad, *uniques]
        self.max_len = max_len
        self.ch2i = {ch: i for i, ch in enumerate(self.alphabet)}

    def encode(self, word: str) -> torch.Tensor:
        indices = [self.ch2i[ch] for ch in word]
        # дополняем до одинакового размера индексом служебного символа
        indices += [self.ch2i[self.pad]] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        pad_indices = torch.nonzero(indices == self.ch2i[self.pad], as_tuple=True)[0]  # noqa
        if len(pad_indices):
            indices = indices[:pad_indices[0]]  # отрезаем служебные символы
        return "".join(self.alphabet[i] for i in indices)


In [346]:
vocab = Vocab(surnames_df["surname"])
indices = vocab.encode("kovalev")
print(indices, vocab.decode(indices))

tensor([32, 29, 50, 55, 53, 35, 50,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) kovalev


2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины


In [347]:
class SurnamesDataset(Dataset):

    def __init__(self, df: pd.DataFrame, vocab: Vocab, transform: t.Callable = None):
        self.surnames = df["surname"].tolist()

        if transform:
            # 1 раз transform - прохождение эпох быстрее
            size = transform(self.surnames[0]).size()
            self.data = torch.vstack([transform(w) for w in self.surnames]).view(len(self.surnames), *size)
        else:
            self.data = self.surnames
        self.targets = torch.tensor(df["target"], dtype=torch.long)

        self.vocab = vocab
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [348]:
def to_indices(word: str) -> torch.Tensor:
    return vocab.encode(word.lower())


def one_hot(word: str) -> torch.Tensor:
    vectors = torch.zeros(vocab.max_len, len(vocab.alphabet))
    indices = [(i, vocab.ch2i[ch]) for i, ch in enumerate(word.lower())]
    vectors[list(zip(*indices))] = 1
    return vectors


# Зачем 2 датасета?
# - нужно для задания 2.6
# - удобнее работать с интерфейсом Dataset'а (разбиение на выборки, DataLoader)
# - эффективнее - одно преобразование вместо преобразование/эпоха

# датасет для обучения через Embedding
surnames_indices_dataset = SurnamesDataset(surnames_df, vocab, transform=to_indices)
# датасет для обучения без Embedding (слова закодированы one-hot методом)
surnames_one_hot_dataset = SurnamesDataset(surnames_df, vocab, transform=one_hot)
surnames_indices_dataset[0], surnames_one_hot_dataset[0]

((tensor([54, 29, 29, 34, 20, 29,  6, 34,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  tensor(4)),
 (tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0

2.3 Разбить датасет на обучающую и тестовую выборку

In [349]:
torch.manual_seed(0)

train_indices_dataset, test_indices_dataset = train_test_split(surnames_indices_dataset, train_part=0.8)
train_one_hot_dataset, test_one_hot_dataset = train_test_split(surnames_one_hot_dataset, train_part=0.8)
print(len(train_indices_dataset), len(test_indices_dataset))

8784 2196


2.6. Обучить классификатор.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта:
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


In [350]:
class SurnamesClassifier(nn.Module):

    def __init__(
            self,
            vocab: Vocab,
            out_features: int,
            embedding_dim: int = 128,
            use_embedding: bool = True,
            debug: bool = False,
    ):
        super(SurnamesClassifier, self).__init__()
        self.use_embedding = use_embedding
        self.debug = debug

        self.embedding_dim = embedding_dim

        last_conv_out_channels = 64
        adaptive_avg_pool = 8

        # Как же этой модели все это... безразлично
        self.embedding = nn.Embedding(num_embeddings=len(vocab.alphabet), embedding_dim=embedding_dim)
        self.features = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=3),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=64, out_channels=last_conv_out_channels, kernel_size=3),
            nn.BatchNorm1d(num_features=last_conv_out_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        # Единственный полезный (и понятный зачем) слой. Зачем? - Позволяет не думать о размерностях
        self.avgpool = nn.AdaptiveAvgPool1d(adaptive_avg_pool)
        self.classifier = nn.Sequential(
            nn.Linear(last_conv_out_channels * adaptive_avg_pool, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, out_features),
        )

        if self.debug:
            # то же самое, но умеет в вывод размерностей
            self.forward = self._debug_forward
        else:
            self.forward = self._forward

    def _forward(self, x: torch.Tensor):
        if self.use_embedding:
            x = self.embedding(x)
        else:
            # Для эксперимента с one-hot - что будет, если растянуть вектора до размера embedding_dim?
            # Ответ: ничего
            x = F.pad(x, (0, self.embedding_dim - x.size(2), 0, 0), value=0)

        # (batch_size, num_features [embedding_dim], n_tokens)
        x = x.reshape(x.size(0), x.size(2), x.size(1))
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        # Интересно, почему в лекциях иногда используют CrossEntropyLoss, а иногда SoftMax + NLLLoss?
        # Они же эквивалентны: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        return torch.log_softmax(x, dim=1)

    def _debug_forward(self, x: torch.Tensor):
        print("x: ", x.size())
        if self.use_embedding:
            x = self.embedding(x)
            print("embedding: ", x.size())
        else:
            x = F.pad(x, (0, self.embedding_dim - x.size(2), 0, 0), value=0)
            print("pad: ", x.size())

        x = x.reshape(x.size(0), x.size(2), x.size(1))
        print("reshape: ", x.size())
        x = self.features(x)
        print("features: ", x.size())
        x = self.avgpool(x)
        print("avgpool: ", x.size())
        x = torch.flatten(x, 1)
        print("flatten: ", x.size())
        x = self.classifier(x)
        print("classifier: ", x.size())
        return torch.log_softmax(x, dim=1)


In [351]:
torch.manual_seed(0)

common_net = SurnamesClassifier(vocab, len(surnames_labeler.classes_)).to(DEVICE)
# Интересно, почему в лекциях иногда используют CrossEntropyLoss, а иногда SoftMax + NLLLoss?
# Они же эквивалентны: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(common_net.parameters(), lr=0.001)

In [352]:
%%time

# One-Hot представление
common_net.use_embedding = False
_ = common_train(
    epochs=10,
    model=common_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=DataLoader(train_one_hot_dataset, batch_size=8, shuffle=True),
    test_dataloader=DataLoader(test_one_hot_dataset, batch_size=512),
    verbose=500,
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 2.870472  [    0/ 8784]
loss: 1.945963  [ 4000/ 8784]
loss: 1.294574  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.559654, Avg loss: 1.541320 

Epoch 2
--------------------------------
loss: 2.096421  [    0/ 8784]
loss: 1.642692  [ 4000/ 8784]
loss: 0.902121  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.599271, Avg loss: 1.430406 

Epoch 3
--------------------------------
loss: 1.510903  [    0/ 8784]
loss: 1.432329  [ 4000/ 8784]
loss: 1.241052  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.612477, Avg loss: 1.373528 

Epoch 4
--------------------------------
loss: 1.490520  [    0/ 8784]
loss: 0.620664  [ 4000/ 8784]
loss: 0.863073  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.626594, Avg loss: 1.338518 

Epoch 5
--------------------------------
loss: 0.789928  [    0/ 8784]
loss: 0.912860  [ 4000/ 8784]
loss: 1.017525  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.636157, Avg loss: 1.310688 

Epoch 6
--------------------------------
loss: 0.685604  [    0/ 8784]

In [353]:
torch.manual_seed(0)

embeddings_net = SurnamesClassifier(vocab, len(surnames_labeler.classes_)).to(DEVICE)
# Интересно, почему в лекциях иногда используют CrossEntropyLoss, а иногда SoftMax + NLLLoss?
# Они же эквивалентны: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(embeddings_net.parameters(), lr=0.001)

In [354]:
%%time

# Embedding представление
embeddings_net.use_embedding = True
_ = common_train(
    epochs=15,
    model=embeddings_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=DataLoader(train_indices_dataset, batch_size=8, shuffle=True),
    test_dataloader=DataLoader(test_indices_dataset, batch_size=512),
    verbose=500,
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 2.867698  [    0/ 8784]
loss: 1.629042  [ 4000/ 8784]
loss: 1.283911  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.545993, Avg loss: 1.661579 

Epoch 2
--------------------------------
loss: 1.629521  [    0/ 8784]
loss: 0.516663  [ 4000/ 8784]
loss: 1.304486  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.591075, Avg loss: 1.466671 

Epoch 3
--------------------------------
loss: 2.111546  [    0/ 8784]
loss: 1.017953  [ 4000/ 8784]
loss: 1.181585  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.647541, Avg loss: 1.230426 

Epoch 4
--------------------------------
loss: 0.738920  [    0/ 8784]
loss: 0.862744  [ 4000/ 8784]
loss: 2.226277  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.673497, Avg loss: 1.167229 

Epoch 5
--------------------------------
loss: 0.569694  [    0/ 8784]
loss: 1.785706  [ 4000/ 8784]
loss: 0.526713  [ 8000/ 8784]
Test Error: 
 Accuracy: 0.682149, Avg loss: 1.102714 

Epoch 6
--------------------------------
loss: 0.687909  [    0/ 8784]

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [355]:
test_loop(
    dataloader=DataLoader(test_indices_dataset, batch_size=512),
    model=embeddings_net,
    loss_fn=loss_fn,
    device=DEVICE,
);

Test Error: 
 Accuracy: 0.707650, Avg loss: 1.317035 



In [356]:
def inference(
        surname: str,
        target: str,
        model: nn.Module,
        vocab: Vocab,
        labeler: LabelEncoder,
        k: int = 3,
        device: str = "cpu",
):
    x = vocab.encode(surname.lower())
    x = x.to(device)

    pred = model(x.unsqueeze(0))
    pred_proba, pred_label_indices = F.softmax(pred, 1).topk(k, dim=1)
    pred_labels = labeler.inverse_transform(pred_label_indices.squeeze().cpu())

    predicts = ", ".join(
        [f"{label} ({prob:.2f})" for (label, prob) in zip(pred_labels, pred_proba.squeeze())]
    )
    print(f"Surname : {surname}")
    print(f"True    : {target}")
    print(f"Predicts: {predicts}\n")

In [357]:
students = [
    "Alexandrova",
    "Baranov",
    "Brusova",
    "Volkova",
    "Kovalev",
    "Kostyuchenko",
    "Kuzin",
    "Likhachev",
    "Telitsyn",
    "Ustimova",
    "Khamikoeva",
]
for surname in students:
    inference(
        surname=surname,
        target="Russian",
        model=embeddings_net,
        vocab=vocab,
        labeler=surnames_labeler,
        device=DEVICE,
    )

Surname : Alexandrova
True    : Russian
Predicts: Russian (0.51), English (0.49), Czech (0.00)

Surname : Baranov
True    : Russian
Predicts: Russian (1.00), English (0.00), Czech (0.00)

Surname : Brusova
True    : Russian
Predicts: German (0.22), Russian (0.16), English (0.14)

Surname : Volkova
True    : Russian
Predicts: Russian (0.96), Czech (0.04), Polish (0.00)

Surname : Kovalev
True    : Russian
Predicts: Russian (1.00), Czech (0.00), English (0.00)

Surname : Kostyuchenko
True    : Russian
Predicts: Russian (1.00), English (0.00), Arabic (0.00)

Surname : Kuzin
True    : Russian
Predicts: Russian (0.73), Chinese (0.19), Czech (0.04)

Surname : Likhachev
True    : Russian
Predicts: Russian (1.00), Czech (0.00), Polish (0.00)

Surname : Telitsyn
True    : Russian
Predicts: Russian (0.82), English (0.18), Scottish (0.01)

Surname : Ustimova
True    : Russian
Predicts: Russian (1.00), Japanese (0.00), Czech (0.00)

Surname : Khamikoeva
True    : Russian
Predicts: Russian (1.00), 

**Вывод:** использование Embedding позволило увеличить точность модели.

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

In [358]:
# NLTK теги частей речи отличаются от WORDNET тегов
def get_pos(word: str) -> str:
    tag = nltk.pos_tag([word])[0][1]
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


STOPWORDS = set(stopwords.words("english"))


def preprocess_review(text: str) -> str:
    text = text.lower()
    # удаляем все символы кроме букв латинского алфавита
    text = re.sub(r"[^a-z]", repl=" ", string=text, flags=re.MULTILINE)

    lemmatizer = nltk.WordNetLemmatizer()
    words = []
    for word in nltk.word_tokenize(text):
        if word not in STOPWORDS:  # удаляем стоп-слова до лемматизации - так можно чуть-чуть сэкономить
            lemma = lemmatizer.lemmatize(word, pos=get_pos(word))
            # удаляем стоп-слова, наивное предположение - не брать леммы короче 3-х символов дало значительный прирост точности
            if lemma not in STOPWORDS and len(lemma) > 2:
                words.append(lemma)

    return " ".join(words)

In [359]:
class ReviewsDataset(Dataset):

    def __init__(self, positive_path: Path, negative_path: Path, seed: int = None):
        self.positive_path = positive_path
        self.negative_path = negative_path
        self.positive_reviews = self.read_reviews(positive_path, preprocess_review)
        self.negative_reviews = self.read_reviews(negative_path, preprocess_review)

        data = self.positive_reviews + self.negative_reviews
        targets = torch.cat([torch.ones(len(self.positive_reviews)), torch.zeros(len(self.negative_reviews))])

        if seed is not None:
            torch.manual_seed(seed)
        indices = torch.randperm(len(data))

        self.data = [data[i] for i in indices]
        self.targets = targets[indices].to(torch.long)

    @staticmethod
    def read_reviews(path: Path, process: t.Callable[[str], str]) -> list[str]:
        reviews = []
        with open(path) as f:
            for review in f.readlines():
                review = process(review)
                if review:
                    reviews.append(review)
        return reviews

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.targets[index]


In [363]:
# кажется, файлы с отзывами были перепутаны:
# хорошие отзывы находились в файле negative_reviews.txt - переименовал.
reviews_path = DATA_DIR / "polarity"
reviews_dataset = ReviewsDataset(
    reviews_path / "positive_reviews.txt",
    reviews_path / "negative_reviews.txt",
    seed=0,
)
len(reviews_dataset), reviews_dataset[0]

(10660, ('play less like come age romance infomercial', tensor(0)))

In [364]:
torch.manual_seed(0)

train_reviews, test_reviews = train_test_split(reviews_dataset, train_part=0.8)
len(train_reviews), len(test_reviews)

(17056, 2132)

In [365]:
class ReviewsVocab:
    pad = "<PAD>"
    unknown = "<UNK>"

    def __init__(self, reviews: t.List[str]):
        uniques = set()
        max_len = 0
        for review in reviews:
            words = nltk.word_tokenize(review)
            uniques.update(words)
            max_len = max(len(words), max_len)

        self.alphabet = [self.pad, self.unknown, *uniques]
        self.max_len = max_len

        w2i = {w: i for i, w in enumerate(self.alphabet)}
        # если ключ отсутствует, будет возвращена 1 - индекс служебного символа
        self.w2i = defaultdict(lambda: 1, w2i)

    def __len__(self):
        return len(self.alphabet)

    @lru_cache(maxsize=8192)  # сомнительная эффективность? Ну да
    def encode(self, review: str) -> torch.Tensor:
        indices = [self.w2i[w] for w in nltk.word_tokenize(review)]
        indices += [self.w2i[self.pad]] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        pad_indices = torch.nonzero(indices == self.w2i[self.pad], as_tuple=True)[0]  # noqa
        if len(pad_indices):
            indices = indices[:pad_indices[0]]
        return " ".join(self.alphabet[i] for i in indices)


In [366]:
vocab = ReviewsVocab([review for review, _ in train_reviews])
print(f"alphabet: {len(vocab)}", f"longest: {vocab.max_len}")
encoded = vocab.encode("this is a neutral review")
encoded, vocab.decode(encoded)

alphabet: 13410 longest: 29


(tensor([    1,     1,     1, 12087,   667,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]),
 '<UNK> <UNK> <UNK> neutral review')

2.2. Обучите классификатор.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`
    - подберите адекватную размерность вектора эмбеддинга:
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


In [367]:
class ReviewsClassifier(nn.Module):
    LAST_CONV_OUT_CHANNELS = 64
    ADAPTIVE_AVG_POOL = 8

    def __init__(self, num_embeddings: int, embedding_dim: int):
        super(ReviewsClassifier, self).__init__()

        # Как же этой модели все это... безразлично
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
        self.features = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=self.LAST_CONV_OUT_CHANNELS, kernel_size=2),
            nn.BatchNorm1d(num_features=self.LAST_CONV_OUT_CHANNELS),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        # Единственный полезный (и понятный зачем) слой. Зачем? - Позволяет не думать о размерностях
        self.avgpool = nn.AdaptiveAvgPool1d(self.ADAPTIVE_AVG_POOL)
        self.classifier = nn.Sequential(
            nn.Linear(self.LAST_CONV_OUT_CHANNELS * self.ADAPTIVE_AVG_POOL, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 2),
        )

    def forward(self, x: torch.Tensor):
        x = self.embedding(x)
        x = x.reshape(x.size(0), x.size(2), x.size(1))
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


def collate(batch: t.List[t.Tuple[str, torch.Tensor]]) -> t.Tuple[torch.Tensor, torch.Tensor]:
    xs, ys = [], []
    for x, y in batch:
        xs.append(vocab.encode(x))
        ys.append(y)
    return torch.vstack(xs), torch.hstack(ys)

In [368]:
torch.manual_seed(0)

net = ReviewsClassifier(num_embeddings=len(vocab), embedding_dim=128).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.000914092001)  # а почему нет?
# будет изменять lr = lr * factor, если на протяжении patience эпох ошибка не менялась более чем на threshold
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode="min",
    patience=5,
    factor=0.333333,
    min_lr=0.000001,
    threshold=0.001,
    verbose=True,
)

# Однако batch_size - очень важно. При 8 и 64 застреваем в плохом оптимуме, 32 тоже не очень...
# 22 > Embedding, Conv1d, BatchNorm1d, ReLU, MaxPool1d, AdaptiveAvgPool1d, Dropout, lr_scheduler, optimizer
train_dataloader = DataLoader(train_reviews, batch_size=22, collate_fn=collate, shuffle=True)
test_dataloader = DataLoader(test_reviews, batch_size=512, collate_fn=collate)

In [369]:
%%time

_ = common_train(
    epochs=20,
    model=net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    lr_scheduler=lr_scheduler,
    verbose=150,
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 0.681000  [    0/17056]
loss: 0.677909  [ 3300/17056]
loss: 0.694153  [ 6600/17056]
loss: 0.693084  [ 9900/17056]
loss: 0.693252  [13200/17056]
loss: 0.691192  [16500/17056]
Test Error: 
 Accuracy: 0.505159, Avg loss: 0.693103 

Epoch 2
--------------------------------
loss: 0.689763  [    0/17056]
loss: 0.689104  [ 3300/17056]
loss: 0.691192  [ 6600/17056]
loss: 0.677704  [ 9900/17056]
loss: 0.706161  [13200/17056]
loss: 0.715148  [16500/17056]
Test Error: 
 Accuracy: 0.532833, Avg loss: 0.690566 

Epoch 3
--------------------------------
loss: 0.688084  [    0/17056]
loss: 0.691872  [ 3300/17056]
loss: 0.667145  [ 6600/17056]
loss: 0.579101  [ 9900/17056]
loss: 0.543175  [13200/17056]
loss: 0.547473  [16500/17056]
Test Error: 
 Accuracy: 0.650563, Avg loss: 0.625727 

Epoch 4
--------------------------------
loss: 0.490492  [    0/17056]
loss: 0.324382  [ 3300/17056]
loss: 0.190905  [ 6600/17056]
loss: 0.582821  [ 9900/17056]
loss: 0.239

У модели явная проблема с переобучением. После 3-ей эпохи тестовая ошибка начала увеличиваться,
в то же время ошибка на обучающей выборки быстро приблизилась к 0.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [370]:
y_test, y_pred = get_y_test_y_pred(net, test_dataloader, DEVICE)

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=["negative", "positive"],
))

              precision    recall  f1-score   support

    negative       0.76      0.67      0.71      1061
    positive       0.71      0.79      0.75      1071

    accuracy                           0.73      2132
   macro avg       0.73      0.73      0.73      2132
weighted avg       0.73      0.73      0.73      2132



In [371]:
def inference(
        review: str,
        target: str,
        model: nn.Module,
        vocab: ReviewsVocab,
        target_names: list[str],
        device: str = "cpu",
):
    x = vocab.encode(preprocess_review(review))
    x = x.to(device)

    pred = model(x.unsqueeze(0))
    pred_proba, pred_label_idx = F.softmax(pred, 1).max(dim=1)
    pred_label = target_names[pred_label_idx.cpu()]

    print(f"Review : {review}")
    print(f"True   : {target}")
    print(f"Predict: {pred_label} ({pred_proba.item():.2f})\n")

In [372]:
reviews = [
    ("No intrigue, poor character disclosure.", "negative"),
    ("A fascinating story. The actors played their characters perfectly.", "positive"),
]
for review, target in reviews:
    inference(
        review=review,
        target=target,
        model=net,
        vocab=vocab,
        target_names=["negative", "positive"],
        device=DEVICE,
    )
# вот это уверенность

Review : No intrigue, poor character disclosure.
True   : negative
Predict: negative (1.00)

Review : A fascinating story. The actors played their characters perfectly.
True   : positive
Predict: positive (1.00)



**Вывод:** единственная обоснованная, логичная и очень простая идея не включать в датасет слова (после обработки) короче 3-х символов дала ощутимый прирост точности модели.

_Размер батча - самый важный гипер и негипер параметр_

_Выкинул из модели слой свертки из 4-х блоков - она этого не заметила_